 <font size="6">Нейронные сети</font>

#  Ограничения Линейного классификатора

Вспомним материал лекции №2:

Мы применяли линейный классификатор к датасету CIFAR10

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/input_img_scalar_product_add_bias_get_scores.jpg" width="600">




- Линейный классификатор: скалярное произведение
- Лосс - функции: SVM,CrossEntropyLoss
- Градиентный спуск
- Оценка точности Линейного классификатора (0.38) на CIFAR10

Попробуем визуализировать, шаблоны, получающиеся усреднением изображений. 

In [ ]:
from IPython.display import clear_output
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L05/lc_cifar10_weights.txt
clear_output()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Display templates
plt.rcParams["figure.figsize"] = (25, 10)

W = torch.from_numpy(np.loadtxt("lc_cifar10_weights.txt")) # load wegths, shape 3073x10
print(f'Shape with bias: {W.shape}')

# Remove bias
W = W[:-1, :]
print(f'Shape without bias: {W.shape}')

# Denormalize
w_min = torch.min(W)
w_max = torch.max(W)
templates =  255 * (W - w_min) / (w_max - w_min)

# Display templates
labels_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(1, 10, i+1)
    img = templates[:,i].view(3, 32, 32).permute(1, 2, 0).type(torch.uint8)
    plt.imshow(img)
    plt.axis('off')
    plt.title(labels_names[i])

Ограничение этого подхода состоит в том, что поскольку для каждого класса существует один шаблон, он будет сохранять информацию обо всех объектах класса сразу (например, на получившихся шаблонах у лошади две головы, машина красная и т.д.). Такой подход не допускает вариативности, а это будет мешать модели запоминать разные варианты объектов одного класса, что ограничивает точность модели.

##  ХОR — проблема

У линейного классификатора есть существенные ограничения применения. Рассмотрим задачу XOR. На вход подаётся упорядоченный набор из двух чисел согласно таблице истинности xor. Задача линейного классификатора сопоставить этим числам их класс согласно таблице. Графически два входных числа можно изобразить на плоскости как координаты и цветом обозначить их истинный класс. Задача классификатора - построить линию, отделяющую красные точки (класс 0) от зелёных точек (класс 1). Однако, видно, что одной линией это сделать геометрически невозможно.

То есть линейный классификатор уже не может справиться с этой задачей.

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/xor_problem.png"  width="600">

## Проблемы классификации более сложных объектов

Человек узнает на изображении кошку или любой другой объект, руководствуясь целостным представлением о данном объекте на изображении. Такое целостное интуитивное представление об объектах для компьютера напрямую недоступно. С точки зрения компьютера изображение представляет собой не более чем таблицу из чисел, кодирующих цвета всех его пикселей. Небольшое цветное изображение (с красным, зеленым и синим цветовыми каналами) в разрешении $240 \times 320$ для компьютера представлено просто упорядоченным набором из $240 \times 320 \times 3 = 230400 $ целых чисел.

Легко себе представить ситуацию, в которой изображения одного и того же объекта будут значительно отличаться на масштабе отдельных пикселей. Так, например, один и тот же кот может быть представлен на фотографии в различных позах, фотографии могут отличаться условиями освещения, яркостью или контрастностью. Кроме того, на одной из фотографий может быть изображен только фрагмент объекта — скажем, только хвост. Все эти факторы не являются преградой для классификации объектов на изображении человеком и мы хотим потребовать того же и для реализованных на компьютере алгоритмов классификации.


Вот лишь малая часть параметров, от которых зависит точность распознавания линейного классификатора.

Обратите внимание, что на всех изображениях, в том или ином виде присутствуют кошки. При решении задачи классификации, например "кошка/собака", все изображения должны быть определены как "кошка".



<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/problem_classification_difficult_object.png"  width="700">

В дополнение к описанным выше сложностям, следует упомянуть проблему внутриклассовой вариации: мы можем приписывать к одному классу объекты, которые допускают широкий спектр определения. Так, например, мы обобщаем классом "кошка" кошек различных пород, размеров и возрастов. "Хороший" алгоритм классификации должен быть устойчив к внутриклассовой вариации и верно распознавать все возможные варианты объектов.

Один из подходов к решению этой проблемы &mdash; модифицировать модель таким образом, чтобы на выходе у нее было не 10, а 100 шаблонов, позволяющих запоминать разные объекты одного класса и далее использовать эти шаблоны для разбиения объектов на классы.

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/modified_model.png"  width="700">

Реализуем эту модель на основе линейного классификатора из лекции №2:

Применяем к выходам классификатора еще один классификатор. Будет ли работать данная модель?

In [ ]:
x = torch.rand(3072) # random image
W1 = torch.randn(3072, 100) * 0.0001 # without bias 
W2 = torch.randn(100, 10) * 0.0001 # without bias 
scores1 = x.matmul(W1) # matrix multiplication, equivalent x@W1
scores2 = scores1.matmul(W2) # matrix multiplication, of the next classifier

print(f'First classifier shape: {scores1.shape}')
print(f'Second classifier shape: {scores2.shape}')

Нетрудно заметить, что последовательное применение двух классификаторов к входным данным эквивалентно применению одного классификатора с матрицей весов, равной произведению двух матриц весов классификаторов примененных последовательно.

$$scores2 = x\cdot W1\cdot W2$$ 
$$W = W1\cdot W2$$
$$scores2 = x\cdot W$$ 

Для того, чтобы последовательно примененные классификаторы не вырождались в один, необходимо применить нелинейность к их выходам, например, сделаем так, чтобы каждый шаблон, предсказывающий класс объекта воспринимал только часть сигналов с выхода первого классификатора:

In [ ]:
scores1 = x.matmul(W1) 
print(f"\nScores1 {scores1[:8]}") # take the first 8 values for visualization
activations = torch.maximum(torch.tensor(0), scores1) # only values greater than zero
print(f"\nActivations {activations[:8]}" ) # take the first 8 values for visualization
scores2 = activations.matmul(W2)
print(f"\nScores2 {scores2}") 

Нелинейность:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/non_linearity.jpg" width="450">

Такая конструкция называется **функцией активации**. И мы уже пользовались подобной, когда разбирали Cross entropy loss (Softmax)

Приведем код в порядок:

In [ ]:
class NeuralNet():
  def __init__(self):
    self.W1 = torch.randn(3072, 100)*0.0001 
    self.W2 = torch.randn(100, 10)*0.0001 

  def predict(self, x):
    scores1 = x.matmul(self.W1) # Linear
    activations1 = torch.maximum(torch.tensor(0), scores1) # activation ReLU 
    scores2 = activations1.matmul(self.W2) # Linear
    return scores2

x = torch.rand(3072) # image
model = NeuralNet()
scores = model.predict(x) 
print(f'scores: \n {scores}')

Ядром этой операции является скалярное произведение. 



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L05/dot_product.png" width="800">

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/input_weght_out.png"  width="700">


И она соответствует одному слою искусственной нейронной сети (за исключением функции активации).



<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/scalar_product_add_bias.png" width="800">

#  Многослойные сети

По мере развития мощности компьютеров, теоретической базы, появления больших датасетов и метода обратного распространения ошибки, появилась возможность строить более сложные сети - многослойные нейронные сети или же в современном понимании просто **нейронные сети**.

Пример **полносвязной (fully connected network)** нейронной сети с двумя скрытыми слоями:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/nn_fully_connected.png"  width="400">

#  Обучение нейронной сети

Нейронная сеть в процессе **обучения** с учителем (англ. fit) последовательно (в определенном порядке)обрабатывает все объекты из обучающей выборки. Обучающая выборка может быть передана нейронной сети при обучении несколько раз и тогда одна полная экспозиция объектов из обучающей выборки называется **эпохой обучения**.

Обучающую выборку разделяют на две части: непосредственно использующуюся для обучения (анг. train) и тестовую (англ. test), на которой происходит оценка качества обучения. Стратегия разделения на train и test подвыборки может быть произвольной, но при разделении следует заботиться о том чтобы эти подвыборки были "похожи". В ходе обучения ошибка работы модели измеряется на train и test данных для контроля **переобучения** (англ. overfit). В случае возникновения переобучения нейронная сеть начинает терять обобщающую способность, что можно заметить по возникновению роста ошибки на тестовой подвыборке в ходе процесса обучения -- в случае переобучения нейронная сеть просто "зазубривает" примеры из обучающей подвыборки, а не аппроксимирует искомую функциональную зависимость между признаками и целевой переменной.

##  Прямое распространение

Рассмотрим процесс **обучения с учителем** (анг. supervised learning) нейронной сети с прямым распространением сигнала (анг. Feedforward neural network). В ходе такого процесса мы хотим аппроксимировать при помощи нейронной сети функциональную зависимость между некоторым набором входных сигналов (признаков) и соответствующим им набором выходных сигналов (целевой переменной), используя множество эталонных пар входное значение - выходное значение. 

На этапе прямого распространения, нейронной сети на вход подаются сигналы для объектов из тренировочной выборки и при помощи **функции потерь** (анг. loss function) производится количественное сравнение полученных на выходе нейронной сети выходных сигналов с эталонными значениями. Далее, на этапе обратного распространения, данная информация будет использована для постройки параметров сети.

Этап подстройки параметров нейронной сети может следовать после вычисления функции потерь при распространении сигнала от одного примера (online) или же после накоплении информации об отклике сети на пакете из нескольких примеров из обучающей выборки (**batch**).

##  Веса сети

**Нейрон** является базовым элементом строения нейронной сети. У нейрона есть определенное количество "входов", которыми он "подключён" к выходным значениям других нейронов. Нейрон осуществляет суммирование приходящих в него входных значений, причём учитывает значения входов с определенны весовыми коэффициентами (или просто **весами**), которые в определенном смысле характеризуют их значимость. Веса сети являются вещественными числами и могут быть как положительными, так и отрицательными. Именно веса нейрона являются изменяемыми параметрами и они подвергаются подстройке во время обучения нейронной сети.

###  Как вычислить результат работы нейронной сети

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/nn__xor_example.png"  width="600">

Рассмотрим задачу классификации XOR, то есть на вход подадим 1 и 0, и будем ожидать 1 на выходе. Веса сети определим случайным образом:

$I1=1\quad I2=0$

$w_1=0.45\quad  w_2=0.78\quad 
w_3=-0.12\quad  w_4=0.13$


$w_5=1.5\quad  w_6=-2.3$

```python
H1 = I1*W1+I2*W3 = 1*0.45+0*-0.12 = 0.45
H2 = I1*W2+I2*W4 = 1*0.78+0*0.13 = 0.78
```

Для того чтобы значения H1 и H2 не выходили за предельные значения, используется функция активации 

```python
H1_out = sigmoid(H1) = sigmoid(0.45) = 0.61
H2_out = sigmoid(H2) = sigmoid(0.78) = 0.69
```

```python
O1_in = 0.61*1.5+0.69*-2.3=-0.672
O1_out = sigmoid(-0.672)=0.33
```

Ответ нейронной сети O1_out = 0.33, а мы ожидали на выходе 1. О том, как скорректировать веса нужным образом будет рассказано в разделе о методе обратного распространения ошибки.

###  Смещение (bias)

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/why_add_bias_example.png"  width="500">

Рассмотрим простой пример. На вход нейрона подаётся вес, умноженный на входное значение. После применения функции активации, в зависимости от веса, при всевозможных значениях входа мы можем получить следующие графики:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L05/non_bias_problem_plot.png"  width="600">

<center><p><em>Source: <a href="https://codeby.school/blog/informacionnaya-bezopasnost/pogrughenie-v-mashinnoe-obuchenie-chast-1">Immersion in Machine learning</a></p> </em></center>

[A Biased Graph Neural Network Sampler](https://arxiv.org/pdf/2103.01089.pdf)

Но что если мы захотим чтобы при ```x=2``` чтобы сеть выводила ```0```, тогда без веса смещения эту задачу не решить.

Просто изменить крутизну сигмоиды на самом деле не получится - вы хотите иметь возможность сдвинуть всю кривую вправо.

**Смещение** (англ. bias) - это дополнительный коэффициент, прибавляющийся к сумме входов, наличие смещения позволяет сдвинуть функцию активации влево или вправо, что может иметь решающее значение для успешного обучения.

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/add_bias_example.png"  width="500">

Тогда, при разных смещениях мы можем получить сдвинутые функции активации, что способствует лучшему обучению нейронной сети:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L05/after_add_bias_plot.png"  width="600">

<center><p><em>Source: <a href="https://codeby.school/blog/informacionnaya-bezopasnost/pogrughenie-v-mashinnoe-obuchenie-chast-1">Immersion in Machine learning</a></p> </em></center>

##  Метод обратного распространения ошибки

Итак, у нас есть все компоненты для обучения 2-х слойной модели:
- предобработали данные
- умножили их на веса
- применили функцию активации
- снова умножили
- вычислили значение функции потерь
- взяли градиент
- обновили веса .... ?
- оценили точность

А как будем искать градиент?
Во второй лекции мы в ручную считали производную от функции потерь и весов. Так как модель поменялась, придется делать это заново

Для того что бы упростить этот процесс используется формализм под названием "Алгоритм обратного распространения ошибки" или "Backpropagation"


**Метод обратного распространения ошибки (англ. backpropagation)** является эффективным методом вычисления градиента от функционала потерь многослойной нейронной сети. Благодаря данному методу становится практически возможным использование метода градиентного спуска для проведения процедуры обучения. Не смотря на то что первые работы по обучению многослойных перцептронов методом обратного распространения ошибки были опубликованы ещё в 1974 году, значительного развития данная технология получила сравнительно недавно, после появления современных вычислительных ресурсов.

###  Основная идея метода

Метод обратного распространения ошибки явно использует структуру многослойной нейронной сети как сложной функции, применяя цепное правило дифференцирования для вычисления градиента от функции потерь по весам сети. Градиент от функции потерь нейронной сети вычисляется последовательно, при движении по вычислительному графу нейронной сети от её выходов в направлении входов. Именно такой порядок обхода **вычислительного графа** и обуславливает название метода.

### Граф вычислений

Любую нейронную сеть можно представить в виде графа "последовательных действий", где результат вычисляется последовательно, одно действие за другим.

Ранее мы вычисляли производную вручную, и для простой модели это было несложно.

Однако по мере добавления слоев модель может оказаться намного сложнее. Например, так выглядит архитектура AlexNet:


<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/alexnet.png"  width="800">

Алгоритм обратного распространения ошибки позволяет находить градиенты для любого графа вычислений, если описываемая им функция дифференцируема.

В его основе лежит правило взятия производной сложной функции (chain rule)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L05/chain_rule.png"  width="700">

####  Пошаговый разбор метода обратного распространения

Алгоритм, по которому вычисляются веса, можно представить в виде графа.

А общее правило взятия градиентов можно представить следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/rule_for_taking_gradients.png"  width="500">

Рассмотрим следующую функцию:

$$f(x,w)=\frac{1}{1+e^{-(w_0x_0+w_1x_1+w_2)}}$$

Её можно представить в виде простого графа вычислений:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/graph_of_calculation_gradient.png"  width="700">

На примере данной несложной функции рассмотрим алгоритм обратного распространения ошибки и найдём величину её градиента по параметрам $w$.
Нам потребуется вычислить частные производные $\frac{\partial f}{\partial w_0}$, $\frac{\partial f}{\partial w_1}$, $\frac{\partial f}{dw_2}$, $\frac{\partial f}{\partial x_0}$ и $\frac{\partial f}{\partial x_1}$. 

Пусть "веса" $w$ инициализированы значениями $w_0=2\;w_1=-3,\;w_2=-3,$а "входные признаки" $x$ принимают значения $x_0=-1.0,\;x_1=-2.0$.

Делая прямой проход через граф вычислений для данной функции, получаем её значение для заданных $w$ и $x$ равным $f=0.73$:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/forward_pass_example.png"  width="800">

Далее, в соответствии с алгоритмом обратного распространения ошибки, рассчитаем частные производные, пройдясь последовательно по графу вычислений, постепенно накапливая искомое значение для градиента функции:

Для начала посчитаем производную функции $\frac{\partial f}{\partial f}$, которая будет равна единице. Движемся дальше по графу вычислений &mdash; следующая вершина содержит функцию $f(x)=\frac{1}{x}$, производная которой равна $\frac{df}{dx}=-\frac{1}{x^2}$



<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/compute_gradient_1_step.png"  width="800">



В следующем узле находится функция $f(x)=1+x$. Производная от константы $+1$ нулю, то есть производная от производная всего выражения в данном узле равняется просто $\frac{df}{dx}=1$:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/compute_gradient_2_step.png"  width="800">

Третья вершина содержит экспоненту $f(x)=e^x$. Её производная также является экспонентой $\frac{df}{dx}=e^x$:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/compute_gradient_3_step.png"  width="800">

Следующая вершина, четвертая, содержит умножение на константу $f(x)=ax$. Производная равна $\frac{df}{dx}=a$:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/compute_gradient_4_step.png"  width="800">

Двигаясь по графу вычислений мы дошли до узла суммирования, который имеет два входа. Относительно каждого из входов локальный градиент в вершине суммирования будет равен $1$. Так как умножение на единицу не изменит значения входного градиента, всем входам узла суммирования мы можем приписать точно такое же значение входного градиента ($0.2$), что мы имели и для самого узла суммирования. Будем действовать аналогично и со всеми остальными узлами суммирования, что встретятся нам в вычислительном графе.


<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/compute_gradient_5_step.png"  width="800">

Двигаясь далее к началу графа вычислений мы подходим к вершинам умножения. Для такой вершины локальный градиент по отношению к какому-либо из входов просто равен значению оставшегося входа, умноженным на входящий градиент. Точно так же мы можем поступить и с оставшейся второй вершиной умножения, которая привязана к $w_1$ и $x_1$:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/compute_gradient_6_step.png"  width="800">

Так, двигаясь по графу вычислений в обратном направлении от выхода сети к входным признакам, мы последовательно для каждого узла умножаем локальный градиент на входящий градиент, используя цепное правило дифференцирования сложной функции. В описанном примере мы полностью разбили граф вычислений на отдельные элементарные узлы. Разбиение вычислительного графа на элементарные узлы вовсе не обязательно -- мы можем сгруппировать несколько вершин вместе, если они образуют дифференцируемую функцию и рассматривать их совместно.  

Так, в нашем примере, мы можем заметить что вычислительный граф можно свести к двум операциям: получению выражения $w_0x_0+w_1x_1+w_2$ и последующему вычислению от него сигмоидельной функции. Важно отметить, что сигмоида обладает важным свойством: её производная может быть выражена через саму сигмоидалную функцию:

$$\frac{d\sigma{(x)}}{dx}=(1-\sigma{(x)})\sigma{(x)}$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/compute_gradient_join_vertices_sigmoid_example.png"  width="800">

В коде, без использования библиотек подсчёт градиентов можно записать как:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/calculating_gradients_in_code.png"  width="800">

#### Более сложные случаи

#####  Обратное распространение для векторов:

В случае, когда выход сети $z$ не один (многоклассовая классификация, тексты, картинки и тд.), необходимо учитывать значения каждого элемента выходного вектора, для расчёта градиентов.

Матрица Якоби — матрица, составленной из частных производных отображения.

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/backpopagation_with_vectors.png"  width="700">

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/backpropagation_with_vectors_example.png"  width="700">

##### Множественная вершина

Если вход соединен с несколькими вершинами графа или у вершины больше одного выхода.


<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/multiple_vertices_of_graph.png"  width="700">


То в месте ветвления можно создать дополнительную вершину, которая будет соответствовать операции копирования.

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/add_copy_operation_for_vertex.png"  width="700">

Тогда при обратном распространении, градиент можно разделить в соответствии с осуществлённой операцией

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/upstream_gradient.png"  width="700">

Для базовых операций можно выделить следующие шаблоны "разделения" градиента:

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/add_copy_mul_max_gates.png"  width="700">

####  Анимация работы метода обратного распространения ошибки



Таким образом, метод обратного распространения ошибки включает в себя следующие шаги:
* Forward propagation (FP) - прямое распространение сигнала от входа к выходам (без которого не получить вычисленные значения в графе).
* Back propagation (BP) - расчёт величины градиента от выхода ко входам.
* Обновление весов, в зависимости от величины градиента. На анимации буквой $\eta$ обозначен learning rate.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L05/backprop_animation.gif"  width="600">

#### Введение в PyTorch

Практически вся наша работа с этого момента будет осуществляться с помощью [PyTorch](https://pytorch.org/), поэтому необходимо познакомиться с основными концептами, принципами и функциями PyTorch.

Лучший друг в этом, конечно же, [документация](https://pytorch.org/docs/stable/index.html), однако можно привести основные моменты этой библиотеки/фреймворка:

##### Основная сущность - torch.tensor

Поскольку основная сущность, с которой мы работаем, это вектора и матрицы, то для них нужен очень мощный и функциональный класс - [torch.Tensor](https://pytorch.org/docs/stable/tensors.html#torch.Tensor)

Пустой класс тензора:

In [ ]:
import torch
a = torch.Tensor() 

Конструктор класса с заполнением существующими значениями:

In [ ]:
a = torch.tensor([1.1, 2.2, 3.2]) 
a.dtype

Явное указание типа данных:

In [ ]:
a = torch.tensor([1.1, 2.2, 3.2], dtype=torch.float64)
a.dtype

Создание 2-мерного тензора, заполненного единицами (для нулей zeros)

In [ ]:
a = torch.ones(size=(3, 2)) 
a.size()

Создание 2-мерного тензора, заполненного нашим значением


In [ ]:
a = torch.full((3, 2), 5.1) 
a

Транспонирование (изменение порядка осей)

In [ ]:
a = a.T 
a

В библиотеке доступно огромное количество встроенных математических примитивов

In [ ]:
c = torch.atan2(a[0], a[1]) 
c

Почти всё, что есть в numpy, есть в pytorch, например sum():

In [ ]:
c.sum() 

Добавление и удаление осей 

In [ ]:
a = torch.zeros((2, 5, 1, 8))
print("Original tensor size:\n", a.size())

a = a.permute(dims=(2, 0, 3, 1)) # permute dimensions
print("After permute tensor size:\n", a.size())

a = a.squeeze() # delete dimension
print("After squzee tensor size:\n", a.size())

Преобразование torch.tensor в numpy массив

In [ ]:
a.numpy() 

PyTorch позволяет тензору быть представлением существующего тензора. Тензор представления использует те же базовые данные, что и его базовый тензор. Поддержка `view` позволяет избежать явного копирования данных, что позволяет нам выполнять быстрое и эффективное изменение формы, нарезку и операции с элементами.

In [ ]:
a = torch.rand(2, 8)
print("Original tensor:\n", a)
b = a.view(4, 4) # carefully with structured data, reshape/view can transform image to unreadable view 
print("Tensor after view tensor:\n", b)
b += 1
print("Add 1 to tensor:\n", b)

Размещение тензора на GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Cuda available: ", torch.cuda.is_available(), '\n')
a = a.to(device) # tensor to gpu
b = torch.full_like(a, 2).to(device)
c = a * b # compute on gpu (more fast with parallel computing)
c

##### Автоматическое вычисление градиентов

Torch умеет запоминать последовательность операций с нашими тензорами и вычислять градиент.

In [ ]:
x_train = torch.tensor([1., 2., 3., 4.])
y_train = torch.tensor([2., 4., 6., 8.])

W = torch.tensor(1.0, requires_grad=True) 

print (f"W.grad = {W.grad} (before forward pass must be 'None')")

y_pred = W * x_train
criterion = torch.nn.MSELoss()
MSE = criterion(y_pred, y_train)
print(f"MSE = {MSE}")

# backward pass to compute gradient dMSE/dw
MSE.backward()
print (f"W.grad = {W.grad}")

Отсоединение тензора от графа вычислений (используйте при копировании тензора)

In [ ]:
W.detach() 

##### Другие интересные подмодули фреймворка


Torch

* ```torch.nn``` - Модуль для работы с нейронными сетями в стиле ООП
* ```torch.nn.functional``` - То же, что выше, но в функциональном стиле
* ```torch.utils.data``` - Создание датасета, даталоадера
* ```torch.linalg``` - Линейная алгебра
* ```torch.fft``` - Преобразования Фурье
* ```torch.random``` - Рандом


Torchvision - работа с изображениями
* ```torchvision.transforms``` - Трансформации для изображений
* ```torchvision.datasets``` - Учебные датасеты, трансформации
* ```torchvision.models``` - Готовые нейронки для изображений

Torchaudio - работа с аудио.
* Подмодули для аудио

Torchtext - работа с текстом.
* Подмодули для текста

etc.

####  Backprop in PyTorch


Рассмотрим пример реализации шага обратного прохода на примере вычисления квадрата ошибки для линейной регрессии (для простоты не будем рассматривать смещение):

$$y=w\cdot x, \quad при \;x=[1,2,3,4],\;y=[2,4,6,8],\;w=1$$

В данном примере видно, что предсказанный моделью $\hat{y}=[1,2,3,4]$ не совпадает с истинными значениями $y$, и соответственно квадратичная ошибка для такого примера будет $$MSE=\frac{1}{4}\sum_{i=1}^4E_i^2=\frac{1}{4}\sum_{i=1}^4(\hat{y}_i-y_i)^2=\frac{1+4+9+16}{4}=7.5$$

Градиент весов $w$ вычисляется следующим образом, в соответствии с chain rule:

$$\frac{d MSE}{d w} = \frac{\partial MSE}{\partial E}\cdot \frac{\partial E}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial w}$$

Рассчитаем его с использованием PyTorch:



In [ ]:
x_train = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
y_train = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

# This is the parameter we want to optimize -> requires_grad=True
W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)
print (f"W.grad = {W.grad} (before forward pass must be 'None')\n")
# forward pass to compute MSE
y_pred = W * x_train
E = y_pred - y_train
SE = E ** 2
MSE = SE.mean()
print(f"MSE = {MSE}")

# backward pass to compute gradient dMSE/dw
MSE.backward()
print (f"W.grad = {W.grad}")
print (f"E.grad = {E.retain_grad()}")

В данном примере мы произвели следующие расчеты:

$\frac{\partial MSE}{\partial E}=\frac{\sum\partial E^2}{\partial E}=\frac{1}{4}\cdot2\cdot E=\frac{1}{2}*[-1, -2, -3, -4]=[-0.5, -1, -1.5, -2]\quad *-поэлементное\;умножение$

$\frac{\partial E}{\partial \hat{y}}=\frac{\partial (\hat{y}-y)}{\partial \hat{y}}=1$

$\frac{\partial \hat{y}}{\partial w}=\frac{\partial wx}{\partial w}=x=[1, 2, 3, 4]$

$\frac{d MSE}{d w} = \frac{\partial MSE}{\partial E}\cdot \frac{\partial E}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial w}=\sum[-0.5, -1, -1.5, -2]*[1, 2, 3, 4]=-0.5-2-4.5-8=-15$

`MSE.backward()` автоматически вычисляет градиент $\frac{dMSE}{dw}$ при указании `requires_grad=True`. 
Результаты вычислений будут храниться в `W.grad`. Для всех промежуточных переменных градиенты не сохраняются, поэтому попытка обратиться, например, к `E.grad` выдает `None`. 

Также после однократного обратного прохода, в целях экономии памяти, граф, используемый для вычисления градиента, будет удаляться и следующий запуск `MSE.backward()` будет выдавать ошибку:



```
MSE.backward() # Error on second backward call
```



Чтобы сохранить вычислительный граф, для аргумента `retain_graph` функции `bacward()` нужно указать значение `True`. Также может быть полезным сохранять значения градиентов для промежуточных переменных, это делается с помощью функции `tensor.retain_grad()`. В таком случае, значения градиентов, полученные на следующих итерациях обратного распространения ошибки, будут складываться с текущими значениями градиентов.

Градиенты переменных, для которых был указан `retain_graph=True` сохраняются автоматически, чтобы избежать их накопления при многократном итерировании алгоритма обратного распространения, нужно обнулять градиент на каждом шаге с помощью функции `tensor.grad.zero_()`

In [ ]:
x_train = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
y_train = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

# This is the parameter we want to optimize -> requires_grad=True
W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

# forward pass to compute MSE
y_pred = W * x_train
E = y_pred - y_train
E.retain_grad() # Save grads for intermediate tensor E in memory
SE = E**2
MSE = SE.sum().div(4)

print("========== Backprop 1 ==============")
MSE.backward(retain_graph=True)
print (f"dMSE/dE = {E.grad}")
print (f"dMSE/dW = {W.grad}")

print("========== Backprop 2 ==============")
MSE.backward(retain_graph=True)
# Gradients are accumulated
print (f"dMSE/dE = {E.grad}")
print (f"dMSE/dW = {W.grad}")

print("========== Backprop 3 ==============")
W.grad.zero_() # Nullify gradients for W for the next iteration
MSE.backward(retain_graph=True)
# Gradients for W are not accumulated, but not for E
print (f"dMSE/dE = {E.grad}")
print (f"dMSE/dW = {W.grad}")

Итак, мы умеем вычислять градиент $\frac{\partial MSE}{\partial w}$ для нашего примера. Теперь давайте с его помощью оптимизируем веса, используя алгоритм обратного распространения ошибки:

In [ ]:
x_train = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
y_tain = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

# Define model output
def forward(x_train):
    return W * x_train

# Compute MSE loss 
def criterion(y_pred, y_train):
    return ((y_pred - y_train)**2).mean()

print(f'Prediction before training: f(x) = {forward(x_train)}')
print(f'True values: y = {y_train}\n')

# Training
learning_rate = 0.005
num_epochs = 102

for epoch in range(num_epochs):
    # Propagate forward
    y_pred = forward(x_train)

    # Compute MSE loss
    MSE = criterion(y_pred, y_train)

    # Propagate backward, compute gradients
    MSE.backward()

    # Update weights
    with torch.no_grad(): #  We don't want this step to be the part of the computational graph
        W -= learning_rate * W.grad 
    
    # Nullify gradients after updating to avoid their accumulation
    W.grad.zero_()

    if epoch % 10 == 1:
        print(f'epoch {epoch}: w = {W.item():.3f}, loss = {MSE.item():.8f}')

print(f'\nPrediction after training: f(x) = {forward(x_train)}')
print(f'True values: y = {y_train}')

Видно, что наш подход позволяет оптимизировать вес $w$ регрессии из примера и таким образом добиться почти идеального предсказания нашей модели, однако в данном подходе дополнительно можно автоматизировать вычисление функции потерь и обновление параметров с учетом градиента, используя готовые функции потерь из `torch.nn` и оптимизаторы из `torch.optim`.

In [ ]:
import torch.nn as nn

x_train = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
y_tain = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

# Define model output
def forward(x_train):
    return W*x_train


print(f'Prediction before training: f(x) = {forward(x_train)}')
print(f'True values: y = {y_train}\n')

# Training
learning_rate = 0.005
num_epochs = 102

criterion = nn.MSELoss()
optimizer = torch.optim.SGD([W], lr=learning_rate)

for epoch in range(num_epochs):
    # Propagate forward
    y_pred = forward(x_train)

    # Compute MSE loss
    MSE = criterion(y_pred, y_train)

    # Propagate backward, compute gradients
    MSE.backward()

    # Update weights
    optimizer.step()

    # Nullify gradients after updating to avoid their accumulation
    optimizer.zero_grad()

    if epoch % 10 == 1:
        print(f'epoch {epoch}: w = {W.item():.3f}, loss = {MSE.item():.8f}')

print(f'\nPrediction after training: f(x) = {forward(x_train)}')
print(f'True values: y = {y_train}')

###  Преимущества и недостатки метода

В настоящее время метод обратного распространения ошибки фактически стал стандартом при обучении широкого спектра современных архитектур нейронных сетей. Этот метод даёт итерационное решение задачи поиска минимума функционала ошибки, последовательно подстраивая веса нейронной сети в ходе обучения. К сожалению, такой подход не гарантирует что в ходе обучения нейронной сети мы действительно достигнем абсолютного минимума функции потерь. Кроме того, в силу итерационной природы алгоритма, для обучения может потребоваться огромное количество циклов, что иногда приводит к необходимости проводить вычисления непрерывно в течении дней, недель или даже месяцев. Остановимся ещё на некоторых трудностях, которые могут сопровождать метод обратного распространения ошибки:  

* Остановка обучения в локальном минимуме функции потерь

    На функцию потерь нейронной сети можно смотреть как на определенную поверхность в пространстве высокой размерности (размерность пространства соответствует числу весов нейронной сети). Скажем, если бы у нейронной сети было всего два веса, то такая поверхность была бы похожа на земной рельеф.

    Поверхность функции потерь во всех возникающих на практике случаях достаточно сложна и содержит высокоразмерные аналоги холмов, впадин, долин и всевозможных их комбинаций. Применяя градиентный спуск, мы фактически движемся по такой поверхности в направлении самого "крутого" склона вниз, начиная своё движение из какой-то фиксированной точки поверхности. Может так оказаться, что мы начали своё движение по поверхности неподалёку от небольшой впадины (локального минимума функции потерь) и, "закатившись" в него, не сможем больше из него выбраться, даже если совсем неподалёку в пространстве весов сети будет присутствовать значительно более "глубокий" минимум -- все пути из неглубокого локального минимума находятся в направлении противоположном тому, согласно которому мы движемся при методе градиентного спуска. Чтобы "выпрыгнуть" из такого нежелательного локального минимума, может потребоваться кратковременно увеличить скорость обучения (фактически размер шага). Проблема выбора алгоритма задания оптимального шага обучения во время градиентного спуска в общем случае не решена.

* Паралич сети

    Пусть мы обучаем многослойную нейронную сеть, в которой к качестве функции активации нейронов используются сигмоидельные функции (логистическая функция или $\text{tanh(x)}$). Если мы допустим слишком сильное обновление весов у большого числа нейронов, чего можно легко добиться выбором слишком большой величины скорости (шага) обучения, то дальнейшее обучение сети фактически может остановиться. Дело в том, что в области больших по модулю значений аргумента сигмоидальной функция практически не изменяется, а значит её производная слабо отличается от нуля. Так как величина обновления весов на следующем шаге градиентного спуска будет пропорциональная значением этих производных, то оно также будет близко к нулю. Такой переход нейронов в "насыщение" может быть уже необратимым и методу градиентного спуска потребуется неограниченно много времени для возвращения сети в исходное "рабочее" состояние.

##  Функции потерь (loss functions)

Предположим, у нас есть нейронная сеть с некоторыми весами, прежде всего мы должны понять, насколько она точна - то есть наши ожидания соответствуют результату работы нейронной сети. Мы подали на вход нейронной сети изображение, сигналы прошли через наши слои и функции активации вперёд **(forward propagation)**, и на выходе мы имеем некоторый ответ. Как его оценить? Насколько он точен?

Для оценки соответствия полученного результата ожидаемому, используют функции потерь (loss function). Значение функции потерь даёт количественную оценку величины такого соответствия.

Функция потерь в нейронной сети принимает два аргумента:
* вектор значений, который мы считаем априорно верным
* вектор значений конечных выходов модели, который должен соответствовать априорно верному

Важно заметить, что для успешного обучения модели методом градиентного спуска мы должны потребовать от функции потерь дифференцируемости и ограниченности снизу.

[PyTorch Docs](https://pytorch.org/docs/stable/nn.html#loss-functions)

###  Mean squared error

Mean Squared Error (MSE) - это средняя квадратическая ошибка. Данная функция потерь очень популярная, поскольку она проста для понимания и реализации, и в целом работает довольно хорошо. Чтобы рассчитать MSE, вы берете разницу между предсказаниями вашей модели и фактическими значениями, вычитаете их, возводя разницу в квадрат и затем усредняете по всему набору данных.
Результат всегда положительный, независимо от знака предсказанных и истинных значений, и идеальное значение равно 0.0

$$MSE=\frac{1}{n}\sum_{i=1}^n(Y_i - \hat{Y_i})^2$$

```python
torch.nn.MSELoss()
```

In [ ]:
criterion = nn.MSELoss()
input_values = torch.Tensor([0.5, -0.25, 0.75])
print(f'input_values: {input_values}')
target = torch.Tensor([1, 0.25, 0.25])
print(f'target: {target}')
loss_mse = criterion(input_values, target)
print(f'loss_mse: {loss_mse}')

* **Преимущество:** Использование MSE в качестве функции потерь даёт основания ожидать, что обученная с ней модель не имеет сильных "выбросов" в величине ошибки. Любой большой выброс в невязке $|Y_i - \hat Y_i| \ggg 0 $ при вычислении MSE был бы возведён в квадрат и дал бы вовсе огромный вклад в итоговое значение функции потерь.


* **Недостаток:** Как логично следует из описанного выше преимущества  MSE, данная функция потерь в первую очередь сильно штрафует модель за наличие выбросов в предсказаниях.  Для ряда практически важных задач, тем не менее, важнее оказывается наиболее высокая точность работы на абсолютном большинстве входных примеров, нежели отсутствие одиночных выбросов.

###  Mean Absolute Error 

Средняя абсолютная ошибка (MAE) -- это величина, которая измеряет среднюю по всем образцам величину невязки $|Y_i - \hat{Y_i}|$. Не смотря на то что определение этой функции потерь похоже на MSE (MSE loss можно назвать $L_2$ ошибкой, а MAE в этом смысле можно назвать $L_1$ ошибкой), средняя абсолютная ошибка имеет существенно другие свойства. 

$$MAE=\frac{1}{n}\sum_{i=1}^n|Y_i - \hat{Y_i}|$$

```python
torch.nn.L1Loss()
```

In [ ]:
criterion = nn.L1Loss()
input_values = torch.Tensor([0.5, -0.25, 0.75])
print(f'input_values: {input_values}')
target = torch.Tensor([1, 0.25, 0.25])
print(f'target: {target}')
loss_mae = criterion(input_values, target)
print(f'loss_mae: {loss_mae}')

* **Преимущество:** Поскольку при вычислении MAE мы считаем абсолютное значение ошибки, данная метрика не придёт такого большого значения "выбросам", как MSE -- все ошибки учитываются равнозначно в единой линейной шкале.


* **Недостаток:**  Недостаток применения MAE в качестве функции потерь при обучении модели напрямую вытекает из преимуществ. Обученная с MAE модель может показывать хорошие (или даже отличные) результаты в большинстве случае, но на отдельных примерах допускать большую ошибку. Если специфика решаемой задачи не позволяет нам пренебречь такими одиночными большими выбросами, то следует воздержаться от применения данной метрики.

###  Cross entropy

Кроссэнтропия (перекрёстная энтропия) является одной из наиболее часто применимых в практике обучения нейронных сетей функцией потерь. С точки зрения теории информации кроссэнтропия между двумя вероятностными распределениями $p$ и $q$ над одним и тем же вероятностным пространством определяет среднее количество информации (измеренное в количестве бит), необходимое для идентификации одиночного события из вероятностного пространства, если информация о таком событии записывается в виде оптимизированного под "оценочное" распределение $q$, нежели чем с использованием "истинного" распределения $p$.

Интуитивно кроссэнетропию можно понимать в качестве меры близости "оценочного" распределения $q$ к истинному распределению $p$. На практике элементами распределения $p$ является набор ground true значений из обучающей выборки, а элементы оценочного распределения $q$ являются выводами используемой нами модели.

Формально кроссэнтропию можно записать в виде:

$$ H(p,q) = -E_p[\log q]$$

Где:
* $E_p$ - оператор математического ожидания относительно распределения $p$

Однако чаще кроссэнтропию определяют с помощью энтропии и расстояния Кульбака-Лейблера:

$$H(p,q) = H(p) +D_{KL}(p||q)$$

В случае нейронных сетей, где вероятности представлены дискретными выходами, формула превращается в:


$$H(p,q)=-\sum_xp(x)\log q(x)$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/probabiliry_cross_entropy.png"  width="800">



Поскольку чаще всего кроссэнтропию используют после softmax, то в готовых реализациях softmax объединяют с кроссэнтропией

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/softmax_with_cross_entropy.png"  width="700">

CrossEntropyLoss из torch включает в себя Softmax!
```python
torch.nn.CrossEntropyLoss()
```

In [ ]:
criterion = nn.CrossEntropyLoss()
input_values = torch.rand(3, 3)
print(f'input_values: {input_values}')
target = torch.empty(3, dtype=torch.long).random_(3)
print(f'target: {target}')
loss_ce = criterion(input_values, target)
print(f'loss_ce: {loss_ce}')

* **Преимущества:** Важное свойство кроссэнтропии - возможность работать с весами для классов. А значит и возможность применения этой функции потерь при работе с несбалансированным датасетом.
* **Недостатки:** Вычислительная сложность выше, чем MSE или MAE

###  Binary cross entropy

В частном случае, когда количество классов равно двум, и их можно закодировать одним числом: 0 - для первого класса, и 1 для второго, то формулу CrossEntropyLoss можно адаптировать для этого случая:

$$H_p(q)=-\frac{1}{N}\sum_{i=1}^N y_i\cdot log(p(y_i))+(1-y_i)\cdot log(1-p(y_i))$$


        torch.nn.BCELoss


Важной особенностью BCELoss является то, что она ждёт одно число выхода сети и одно число как верный результат. Тут используется не one-hot кодировка для двух классов, а **одно число 0 - первый класс, 1 - второй класс.**

In [ ]:
criterion = nn.BCELoss()
input_values = torch.rand(1)
print(f'input_values: {input_values}')
target = torch.empty(1).random_(2)
print(f'target: {target}')
loss_bce = criterion(input_values, target)
print(f'loss_bce: {loss_bce}')

Если классы "абсолютно полностью" не совпали, то возникает ситуация взятия логарифма от 0, а он не определён и стремится к бесконечности, поэтому берётся "обрезанная бесконечность" равная 100.

Далее, если сэмплов несколько, то по умолчанию берётся среднее по семплам. См. аргумент `reduction`.

In [ ]:
criterion = nn.BCELoss()
input_values = torch.ones((5))
print(f'input_values: {input_values}')
target = torch.zeros(5)
print(f'target: {target}')
loss_bce = criterion(input_values, target)
print(f'loss_bce: {loss_bce}')

###  Итоги

Кросс-энтропия предпочтительнее для *классификации*, в то время как среднеквадратичная ошибка является одним из лучших вариантов для *регрессии*. Это происходит непосредственно из самой постановки задач &mdash; в классификации вы работаете с очень конкретным набором возможных выходных значений, поэтому MSE плохо определен (поскольку он не обладает такого рода знаниями, поэтому наказывает ошибки несовместимым образом). Чтобы лучше понять явления, полезно проследить и понять отношения между ними.

И то, и другое можно рассматривать как оценки максимального правдоподобия, просто с различными предположениями о зависимой переменной.

##  Функции активации

Минимальным функциональным элементом нейронной сети является одиночный нейрон. Нейрон осуществляет две операции:
1. вычисляет взвешенную смещенную суму значений его входов
$$ z=\sum_{i=1}^n w_i \cdot x_i+b=WX+b,$$
где $W$ обозначает вектор весов, $X$ обозначает вектор входных значений, а $b$ задаёт величину смещения.
1. применяет к получившейся величине некоторую нелинейную функцию, называемую функцией активации
$$ z \rightarrow \text{activation} (z)$$


Взвешенная смещенная сумма входов $z$ может принимать на вещественной прямой. Данное значение  передается в функцию активации, которая, как правило, обладает другим множеством возможных значений.

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/neurons_output.png"  width="900">

Идея применения функций активации в структуре искусственных нейронных сетей обусловлена биологической аналогией. Известно, что в биологических нейронных сетях имеется аналог нелинейной функции активации: существует пороговый потенциал, только после достижения которого происходит возбуждение потенциала действие в клетке и, как следствие, распространение сигнала по нейронной сети. 


Именно таким простейшим образом ведёт себя пороговая функция активации, которая использовалась при построении первых искусственных нейронных сетей -- перцептронов:


$f(x) =
\begin{cases}
0, &\text{$x<b$} \\ 
1, &\text{$x\geq b$}
\end{cases}
$


<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/threshold_function_plot.png"  width="300">


Построенная с пороговой функцией активации нейронная сеть обладает ключевым недостатком, не позволяющим фактически использовать данную функцию активации на практике. В силу того что производная функции активации тривиальна прочти всюду на числовой прямой: 

$f'(x) =
\begin{cases}
0, &\text{$x\neq b$} \\ 
?, &\text{$x= b$}
\end{cases}
$

не представляется возможным использовать метод градиентного спуска для оптимизации параметров нейронной сети.

###  Свойства функций активации

Функции активации должны обладать следующими свойствами:

* **Нелинейность:**
Функция активации необходима для введения нелинейности в нейронные сети. Если функция активации не применяется, выходной сигнал становится простой линейной функцией. Неактивированная нейронная сеть будет действовать как линейная регрессия с ограниченной способностью к обучению:
$$\hat{y}=NN(X,W_1,...,W_n)=X\cdot W_1\cdot ...\cdot W_n=X\cdot W$$ 
Только нелинейные функции активации позволяют нейронным сетям решать задачи аппроксимации нелинейных функций:
$$\hat{y}=NN(X,W_1,...,W_n)=\sigma(...\sigma(X\cdot W_1)...\cdot W_n)\neq X\cdot W$$

* **Возможность прохождения градиента:** 
Функции активации должны быть способными пропускать градиент, чтобы было возможно оптимизировать параметры сети градиентными методами, в частности использовать алгоритм обратного распространения ошибки.

###  Различные функции активации

Рассмотрим наиболее популярные функции активации и обсудим их преимущества и недостатки.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L05/popular_activation_functions.png"  width="700">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1911.05187.pdf">AI in Pursuit of Happiness, Finding
Only Sadness: Multi-Modal Facial
Emotion Recognition Challenge</a></p> </em></center>

####  **Логистическая (сигмоидальная)**

Sigmoid (сигмоидальная) для одномерного случая - используется в задачах классификации, в основном после выхода последнего нейрона. Позволяет определить вероятность принадлежности к одному из двух классов (0 или 1)

$$f(x)=\frac{1}{1+e^{-x}}=\frac{e^{x}}{e^{x}+1}=\frac{1}{2}+\frac{1}{2}tanh(\frac{x}{2})$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/activation_function_sigmoid.png"  width="400">

Производная логистической функции:

$$\frac{d}{dx}f(x)=\frac{e^x\cdot (1+e^x)-e^x \cdot e^x}{(1+e^x)^2}=\frac{e^x}{(1+e^x)^2}=f(x)(1-f(x))=f(x)f(-x)$$

Если активационная функция не бинарная (не как пороговая), то для нейрона возможны значения “активирован на 50%”, “активирован на 20%” и так далее. Если активированы несколько нейронов, можно найти нейрон с наибольшим значением активационной функции.

Так как существуют промежуточные значения на выходе нейрона, **процесс обучения проходит более гладко и быстро**, а вероятность появления нескольких полностью активированных нейронов во время тренировки снижается по сравнению со ступенчатой функцией активации (хотя это зависит от того, что вы обучаете и на каких данных).

```python
torch.nn.Sigmoid()
```

In [ ]:
activation = nn.Sigmoid()
input_values = torch.randn(5) * 5
activation_sig = activation(input_values)
print(f'input_values: {input_values}\nactivation_sig: {activation_sig}')

Сигмоида выглядит гладкой и подобна пороговой функции.

**Достоинства:**

Во-первых, сигмоида — нелинейна по своей природе, а комбинация таких функций производит тоже нелинейную функцию. Поэтому мы можем конструировать многослойные сети.

Еще одно достоинство такой функции — она гладкая, следовательно, улучшается гладкость градиента, в отличие от ступенчатой функции.


**Недостатки:**

Насыщение сигмоиды приводит к затуханию градиентов. Крайне нежелательное свойство сигмоиды заключается в том, что при насыщении функции с той или иной стороны (0 или 1), градиент на этих участках становится близок к нулю. Напомним, что в процессе обратного распространения ошибки данный (локальный) градиент умножается на общий градиент. Следовательно, если локальный градиент очень мал, он фактически обнуляет общий градиент. В результате, сигнал почти не будет проходить через нейрон к его весам и рекурсивно к его данным. Кроме того, следует быть очень осторожным при инициализации весов сигмоидных нейронов, чтобы предотвратить насыщение. Например, если исходные веса имеют слишком большие значения, большинство нейронов перейдет в состояние насыщения, в результате чего сеть будет плохо обучаться.

Выход сигмоиды не центрирован относительно нуля. Это свойство является нежелательным, поскольку нейроны в последующих слоях будут получать значения, которые не центрированы относительно нуля, что оказывает влияние на динамику градиентного спуска. Если значения, поступающие в нейрон, всегда положительны (например, $x > 0$ поэлементно в $f = wx + b$), тогда в процессе обратного распространения ошибки все градиенты весов $w$ будут либо положительны, либо отрицательны (в зависимости от градиента всего выражения $f$). Это может привести к нежелательной зигзагообразной динамике обновлений весов. Однако следует отметить, что когда эти градиенты суммируются по пакету, итоговое обновление весов может иметь различные знаки, что отчасти нивелирует описанный недостаток. Таким образом, отсутствие центрирования является неудобством, но имеет менее серьезные последствия, по сравнению с проблемой насыщения.

####  **Гиперболический тангенс**

Гиперболический тангенс по определению близок к логистической функции. Данные функции можно выразить друг через друга просто через масштабное преобразование:

$$f(x)=tanh(x)=\frac{2}{1+e^{-2x}}-1$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/activation_function_tanh.png"  width="500">

Гиперболический тангенс симметричен относительно нуля и может принимать как положительные, так и отрицательные значения. Данное свойство гиперболического тангенса оказывается важным, в частности, при построении рекуррентных нейронных сетей. При использовании в рекуррентных сетях, получаемые на выходе $\tanh (x)$ положительные или отрицательные значения могут не только увеличивать величину скрытого состояния в ячейках памяти, но и уменьшать их. Подробнее с устройством рекуррентных нейронных сетей мы познакомится в последующих лекциях нашего курса.

**Достоинства**: В силу близкого определения, гиперболический тангенс обладает основными достоинствами описанной выше логистической функции. Кроме того, множество значений данной функции активации симметрично относительно нуля ($[-1,1]$). Использование гиперболического косинуса в качестве функции активации хорошо подходит для последовательного соединения полносвязных слоёв нейронной сети.

**Недостатки**: Производная гиперболического косинуса аналогична по структуре производной логистической функции, следовательно при использовании гиперболического косинуса в качестве функции активации мы также можем столкнуться с проблемой затухания градиентов в области насыщения функции.



```python
torch.nn.Tanh()
```

In [ ]:
activation = nn.Tanh()
input_values = torch.tensor([11.1529,  4.3029,  0.5081, -3.8456, -1.9058])
activation_tanh = activation(input_values)
print(f'input_values: {input_values}\nactivation_tanh: {activation_tanh}')

####  **ReLU**

Часто на практике часто применяется функция активации ReLU. Значение данной функции равно нулю для всех отрицательных входных значений или совпадает с входным значением, если оно неотрицательно. Называние ReLU (rectified linear unit), "выпрямитель", связана с электротехнической аналогией -- график вольт-амерной характеристики идеального выпрямительного диода похож на график функции ReLU.


$$relu(x)=max(0,x)$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/activation_function_relu.png"  width="550">

Производная ReLU:

$$\frac{d}{dx}relu(x) =
\begin{cases}
\frac{d}{dx}0, &\text{$x<0$} \\ 
\frac{d}{dx}x, &\text{$x\geq0$}
\end{cases}=
\begin{cases}
0, &\text{$x<0$} \\ 
1, &\text{$x\geq0$}
\end{cases}
$$

```python
torch.nn.ReLU()
```

In [ ]:
activation = nn.ReLU()
input_values = torch.randn(5)
activation_relu = activation(input_values)
print(f'input_values: {input_values}\nactivation_relu: {activation_relu}')

Рассмотрим положительные и отрицательные стороны ReLU.

**Достоинства:** 

Функция Relu не требует проведения вычислений в вещественной арифметике, как того требую сигмоидальная функция или гиперболический тангенс. Кроме того, производной функции ReLU является просто кусочно-постоянная функция и также может быть вычислена крайне эффективно. Это приводит к тому, что количество необходимых вычислительных ресурсов для обучения нейронной сети с использованием RuLu активаций оказывается значительно ниже, нежели чем при использовании рассмотренных выше логистической функции или $\text{tanh} (x)$. Необходимо также отметить, что использование ReLu не приводит к эффекту насыщения градиента.


**Недостатки:**

Иногда при использовании ReLU в качестве функции активации мы можем столкнуться с нежелательным эффектом отключения ("умирания") отдельных нейронов. Механизм данного явления связан с возможностью получения на выходе функции активации нулевого значения при широком диапазоне входных сигналов -- любая отрицательная линейная комбинация входных значений с весами нейрона будет преобразованная ReLU в ноль. Если при текущем обновлении весов нейрона изменение может оказаться слишком большим (например, при выборе слишком высокой скорости обучения), новая конфигурация весов нейрона будет при любых входных значениях приводить к отрицательной линейной комбинации и, как следствие, тожественно равной нулю активации рассматриваемого нейрона. Такой нейрон также тождественно обратит в ноль и проходящий через него локальный градиент при обучении сети методом обратного распространения ошибки, что сделает практически невозможным возвращение нейрона в "рабочее" состояние. 

#### **Leaky ReLU**

Leaky ReLU (ReLU с «утечкой», название также обусловлено электротехнической аналогией) является простейшей модификацией описанной выше ReLU, призванной исправить проблему "умирания" отдельных нейронов. В отличие от ReLU данная функция не равна константе $0$ при всех отрицательных входных значениях, а реализует в этой области линейную зависимость с небольшим угловым коэффициентом (например, с угловым коэффициентом $10^{-2}$). 

$$lrelu(x)=max(0.01x,x)$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/activation_function_leaky_relu.png"  width="500">

Производная leaky ReLU:

$$\frac{d}{dx}lrelu(x)=\frac{d}{dx}max(0.01x,x)=\begin{cases}
\frac{d}{dx}0.01x, &\text{$x<0$} \\ 
\frac{d}{dx}x, &\text{$x\geq0$}
\end{cases}=
\begin{cases}
0.01, &\text{$x<0$} \\ 
1, &\text{$x\geq0$}
\end{cases}$$

```python
torch.nn.LeakyReLU()
```

In [ ]:
activation = nn.LeakyReLU(0.01)
input_values = torch.randn(5)
activation_lrelu = activation(input_values)
print(f'input_values: {input_values}\nactivation_lrelu: {activation_lrelu}')

**Достоинства**: Сохраняет достоинства ReLU, при этом не страдает от проблемы "умирания" 

**Недостатки**: Некоторые исследователи сообщают об успешном применении данной функции активации, но результаты не всегда стабильны.

####  **GELU (Gaussian Error Linear Unit)**

Функция активации, используемая в трансформерах: Google BERT и OpenAI GPT-2.

$$GELU(x)=xP(X\leq x)=x\Phi(x)=x\cdot \frac{1}{2}[1+erf(\frac{x}{\sqrt{2}})]$$
$$erf(x)=\frac{2}{\sqrt{\pi}}\int_0^xe^{-t^2}dt$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/activation_function_gelu.png"  width="550">

**Достоинства**: State-of-the-art функция активации в задачах NLP

In [ ]:
activation = nn.GELU()
input_values = torch.randn(5) * 5
activation_gelu = activation(input_values)
print(f'input_values: {input_values}\nactivation_gelu: {activation_gelu}')

### Визуализация функций активации:




<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/animated_activation_functions.gif"  width="600">

[Activation Functions Explained](https://mlfromscratch.com/activation-functions-explained/)

#  Пример простой сети на датасете mnist

Загрузим датасет

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from IPython.display import clear_output

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5), (0.5))])

train_set = MNIST(root='./MNIST', train=True, download=True, transform=transform)
test_set = MNIST(root='./MNIST', train=False, download=True, transform=transform)

batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

clear_output()

Набор данных MNIST (англ. Modified National Institute of Standards and Technology) представляет собой модельный набор данных с изображениями рукописных цифр, который часто используется для обучения и оценки качества работы ML моделей классификации изображений.

Датасет состоит из $60000$ тренировочных и $10000$ тестовых изображений. Все изображ ения имеют одинаковый квадратный размер $28 \times 28$ пикселей и единственный цветовой канал. Рукописные изображения цифр имеют равный размер и располагаются в центре кадра. Кроме того, все изображения имеют одинаковую яркость. Всего датасет содержит 10 классов изображений -- цифры от $0$ до $9$.

In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np

x_train = train_set.data.numpy() # images
y_train = train_set.targets.numpy() # labels
labels_names = list(map(str, range(10)))
plt.figure(figsize = (20.0, 20.0))  
for i in range(10):  # for all classes (0 to 9)
  label_indexes = np.where(y_train == i)[0] # get indexes for each class 
  index = np.random.choice(label_indexes)
  img = x_train[index]

  plt.subplot(1, 10, i + 1)  
  plt.title(labels_names[i])  
  plt.axis("off")  
  imshow(img,cmap='gray')  

В PyTorch для создания нейронных сетей требуется отнаследоваться от класса nn.Module и переопределить метод forward, в который подаются входные данные, и ожидаются выходные данные.

[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

В класс мы добавляем две переменные, два слоя Linear. Linear -это слой, позволяющий, умножить веса на входной вектор и добавить смещение. Первый параметр - размер входного вектора, второй - размер выходного.

В методе forward мы указываем последовательность применения операций для получения результата. Сначала изменим представление входного вектора, чтобы от изменения batch_size у нас ничего не сломалось.

Далее идёт первый слой, после него функция активации relu и второй слой, возвращающий вектор длиной 10, означающий принадлежность к одному из классов.

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128) 
        self.fc2 = nn.Linear(128, 10)
        self.activation = nn.ReLU()

    def forward(self, x): # Called inside __call__ method
        x = x.view(-1, 28*28) # "reshape" image to vector
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

Наследование от nn.Module позволяет обращаться к параметрам сети через model.parameters(), а так же упростить много других вещей. **Это критически удобно для обучения сетей**

Второй вариант последовательного выполнения слоёв сети - [nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) - объединение слоёв, вход одного слоя попадает в следующий.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
                                     nn.Linear(28*28, 128),
                                     nn.ReLU(),
                                     nn.Linear(128, 10)
                                    )
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.layers(x)
        return x

In [ ]:
model = Net()

Определим нашу лосс-функцию

In [ ]:
criterion = nn.CrossEntropyLoss()

Определим оптимайзер для нашей сети. Оптимайзер, в библиотеках для нейронных сетей - это сущность, осуществляющая градиентный спуск. Подробнее об оптимайзере будет рассказано в дальнейших лекциях.

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

Обучим сеть десять эпох

In [ ]:
from tqdm import trange

num_epochs = 10
loss_hist = [] # for plotting
epochs = trange(num_epochs)
for epoch in epochs:
    hist_loss = 0
    for _, batch in enumerate(train_loader, 0): # get batch
        # parse batch 
        imgs, labels = batch
        # sets the gradients of all optimized tensors to zero.
        optimizer.zero_grad()
        # get outputs
        pred = model(imgs)
        # calculate loss
        loss = criterion(pred, labels)
        # calculate gradients
        loss.backward()
        # performs a single optimization step (parameter update)
        optimizer.step()
        hist_loss += loss.item()
    loss_hist.append(hist_loss / len(train_loader))
    epochs.set_description(f"Epoch={epoch}  loss={loss_hist[epoch]:.4}")

In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(range(num_epochs), loss_hist)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.show()

Давайте посчитаем accuracy.

Помните, что accuracy является частным случаем метрики качества, поэтому она не реализована в pytorch и поэтому с ней нужно быть аккуратней. Функция ниже подходит только для задачи классификации при one-hot энкодинге, когда классов минимум 2.

In [ ]:
def calaculate_accuracy(model, data_loader):
    correct, total = 0, 0 
    with torch.no_grad(): 
        for batch in data_loader: # get batch
            imgs, labels = batch # parse batch
            pred = model(imgs) # get output
            _, predicted = torch.max(pred.data, 1) # get predicted class
            total += labels.size(0) # all examples
            correct += (predicted == labels).sum().item() # correct predictions 
    return correct / total 

In [ ]:
acc_train = round(calaculate_accuracy(model, train_loader), 2)
print(f"Accuracy train = {acc_train}")
acc_test = calaculate_accuracy(model, test_loader)
print(f"Accuracy test = {acc_test}")

Поэтапно пропустим тестовые изображения через модель и визуализируем результат

In [ ]:
# get batch
imgs, labels = next(iter(test_loader))
print('imgs shape: ', imgs.shape)

In [ ]:
# get output
pred = model(imgs)
print('pred shape: ', pred.shape)

In [ ]:
# First sample in prediction batch
pred[0]

In [ ]:
# Calculate probabilities
nn.Softmax(dim=0)(pred[0].detach())

In [ ]:
# remove axis
imgs = torch.reshape(imgs, (64, 28, 28))
print('imgs shape(after reshape): ', imgs.shape)

In [ ]:
# take 10 first images
imgs = imgs[:10]
print('imgs shape: ', imgs.shape)

In [ ]:
pred = pred[:10].detach()
print('Prediction(1 sample):\n', pred[0])
digits = np.argmax(pred.numpy(), axis=1)
print('Predicted class: ', digits[0])

Визуализируем изображения, подпишем предсказанное и истинное значение:

In [ ]:
plt.figure(figsize = (25.0, 25.0))
for i in range(10):
  img = imgs[i]

  plt.subplot(1, 10, i + 1)
  plt.title('pred: ' + str(digits[i]) + ' real: '+str(labels[i].numpy())) # predicted and real values
  plt.axis("off")
  imshow(img.numpy(),cmap='gray') 


 <font size="6">Ссылки:</font>

[StatSoft. Радиальная базисная функция](http://statsoft.ru/home/textbook/modules/stneunet.html#radial)

[Важность функции потери в машинном обучении](https://towardsdatascience.com/importance-of-loss-function-in-machine-learning-eddaaec69519#:~:text=At%20its%20core%2C%20a%20loss,to%20minimize%20the%20loss%20function.)

[Understanding Categorical Cross-Entropy Loss, Binary Cross-Entropy Loss, Softmax Loss, Logistic Loss, Focal Loss and all those confusing names](https://gombru.github.io/2018/05/23/cross_entropy_loss/)

[Функции активации нейросети: сигмоида, линейная, ступенчатая, ReLu, tahn](https://neurohive.io/ru/osnovy-data-science/activation-functions/)

[Объясненные современные функции активации: GELU, SELU, ELU, ReLU и другие](https://mlfromscratch.com/activation-functions-explained/)